In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [7]:
from src.inference import get_feature_store

In [8]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 19:12:53,833 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:12:53,836 INFO: Initializing external client
2025-03-03 19:12:53,837 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-03 19:12:54,417 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
Fetching data from 2025-02-03 00:12:53.830559+00:00 to 2025-03-03 23:12:53.830559+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.14s) 


In [9]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 01:00:00+00:00,2,0
1,2025-02-03 02:00:00+00:00,2,0
2,2025-02-03 03:00:00+00:00,2,0
3,2025-02-03 04:00:00+00:00,2,0
4,2025-02-03 05:00:00+00:00,2,0
...,...,...,...
173687,2025-03-03 16:00:00+00:00,263,89
173688,2025-03-03 17:00:00+00:00,263,114
173689,2025-03-03 18:00:00+00:00,263,116
173690,2025-03-03 19:00:00+00:00,263,102


In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173692 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         173692 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  173692 non-null  int32                  
 2   rides               173692 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [11]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [12]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173692 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         173692 non-null  datetime64[us]
 1   pickup_location_id  173692 non-null  int32         
 2   rides               173692 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [13]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [14]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,3,0,0,3,5,1,...,0,1,0,0,0,16,0,0,255,2025-02-20 23:00:00
1,2,1,0,0,1,1,0,1,0,1,...,0,0,1,0,0,2,0,0,218,2025-02-28 19:00:00
2,369,273,249,21,251,348,17,373,286,251,...,32,324,393,166,35,89,335,118,237,2025-02-16 06:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-04 18:00:00
4,0,0,0,0,0,0,1,0,1,0,...,0,1,0,1,0,1,1,0,56,2025-02-26 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-03-03 18:00:00
247,0,2,0,0,4,0,1,0,0,2,...,2,0,0,0,2,0,0,0,71,2025-02-16 10:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-27 23:00:00
249,0,1,1,1,1,2,0,4,0,1,...,1,1,2,4,1,0,0,0,168,2025-02-28 15:00:00


In [15]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 19:14:45,103 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:14:45,106 INFO: Initializing external client
2025-03-03 19:14:45,106 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-03 19:14:45,861 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
Fetching data from 2025-02-03 00:14:45.103602+00:00 to 2025-03-03 23:14:45.103602+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.85s) 


In [16]:
current_date

Timestamp('2025-03-04 00:14:45.103602+0000', tz='Etc/UTC')

In [17]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 01:00:00
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,3,2025-03-03 01:00:00
2,0,1,0,1,0,2,3,3,4,6,...,6,8,3,2,1,1,3,1,4,2025-03-03 01:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 01:00:00
4,0,0,0,1,0,0,4,2,2,1,...,4,1,5,3,4,0,1,0,7,2025-03-03 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-03 01:00:00
247,2,1,0,1,0,1,2,2,1,0,...,2,3,0,3,3,0,0,0,260,2025-03-03 01:00:00
248,0,0,1,0,2,2,5,11,10,7,...,42,35,32,13,10,7,5,0,261,2025-03-03 01:00:00
249,2,2,0,7,14,52,136,149,139,98,...,65,72,38,26,23,7,9,5,262,2025-03-03 01:00:00


In [18]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 19:15:02,658 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:15:02,660 INFO: Initializing external client
2025-03-03 19:15:02,660 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:15:03,196 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683


In [19]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [20]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,5.0
3,6,0.0
4,7,0.0
...,...,...
246,259,0.0
247,260,1.0
248,261,2.0
249,262,3.0


In [21]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 138,  48, 249, 186,  79, 114, 230, 246,  68], dtype=int32)